In [1]:
import numpy as np


# Plotting
import matplotlib.pyplot as plt
from matplotlib import scale as mscale
from matplotlib.transforms import Transform
from matplotlib.ticker import FixedLocator

import seaborn as sns

# CLASS

from classy import Class

In [2]:
# Creates an instance of CLASS
cosmo = Class()
CLASS_INPUT={}
raw_cl={}

In [3]:
# Prepare Inputs
CLASS_INPUT['Default']={}
CLASS_INPUT['Default']['output']='tCl,pCl'

In [4]:
cosmo.struct_cleanup()
cosmo.empty()
cosmo.set(CLASS_INPUT['Default'])
cosmo.compute()
raw_cl['Default']=cosmo.raw_cl()
cosmo.struct_cleanup()
cosmo.empty()

In [5]:
raw_cl['Default']

{'tt': array([0.00000000e+00, 0.00000000e+00, 1.56601911e-10, ...,
        9.46211901e-18, 9.41637632e-18, 9.37090231e-18]),
 'ee': array([0.00000000e+00, 0.00000000e+00, 7.65399293e-15, ...,
        3.66635324e-19, 3.67907705e-19, 3.69187390e-19]),
 'te': array([ 0.00000000e+00,  0.00000000e+00,  5.04243448e-13, ...,
        -2.21451894e-19, -2.23353523e-19, -2.25412307e-19]),
 'bb': array([0., 0., 0., ..., 0., 0., 0.]),
 'ell': array([   0,    1,    2, ..., 2498, 2499, 2500])}

In [ ]:


wtinv=(11*10**-8)**2 # K^2 likely need to be changed, but can't find definitive source
weinv=(25*10**-8)**2 # K^2 likely need to be changed, but can't find definitive source
B_l_inv=np.exp((np.arange(2,2501)*0.16*np.pi/180)**2)  #likely need to be changed, but can't find definitive source
err_t_l=wtinv*B_l_inv;
err_e_l=weinv*B_l_inv;
fsky=0.8;
cov_names_to_idx={'t':0,'e':1,'c':2}
Normalizing=1/(2*raw_cl['Default']['ell'][2:]+1)/fsky;
num_ell=len(raw_cl['Default']['ell'])-2
cov_l={};
cov_l['tt']=2*Normalizing*(raw_cl['Default']['tt'][2:]+err_t_l)**2
cov_l['ee']=2*Normalizing*(raw_cl['Default']['ee'][2:]+err_e_l)**2
cov_l['cc']=Normalizing*(raw_cl['Default']['te'][2:]**2+(raw_cl['Default']['tt'][2:]+err_t_l)*(raw_cl['Default']['ee'][2:]+err_e_l))
cov_l['te']=cov_l['et']=2*Normalizing*(raw_cl['Default']['te'][2:])**2
cov_l['tc']=cov_l['ct']=2*Normalizing*raw_cl['Default']['te'][2:]*(raw_cl['Default']['tt'][2:]+err_t_l)
cov_l['ec']=cov_l['ce']=2*Normalizing*raw_cl['Default']['te'][2:]*(raw_cl['Default']['ee'][2:]+err_e_l)

Cov_mats_l=np.empty((num_ell,3,3))
for row_name,row_idx in cov_names_to_idx.items():
    for col_name,col_idx in cov_names_to_idx.items():
        Cov_mats_l[:,row_idx,col_idx]=cov_l[col_name+row_name]
        
Fisher_cl=np.linalg.inv(Cov_mats_l)

In [ ]:
dsg_dict={'dsg_log10a_vals':np.concatenate(([-15],np.arange(-4,-1.9,.5),[1])),
    'dsg_w_vals':np.array([-0.99,-0.9,-0.3,0,0.3,0.9,0.99]),
    'dsg_alpha' : 0.0000006268026419446873,
    'dsg_c_eff2':1/3.,
    'nap' : 'y',
    'dsg_c_vis2':1/3.}
dsg_test_params=classy_params(**{**dsg_dict,**{'h':0.67556}})
